# Prospecção de Dados (Data Mining) DI/FCUL - Project

## Project (MC/DI/FCUL - 2024)

### GROUP:`09`

* Afonso Gama, 55857 - x Hours
* Eduardo Carneiro, 62515 - 6 Hours
* Guilherme Rosario, 62543 - x Hours
* Marco Viana, 62550 - x Hours

# TODO list:
- Compare which molecules are similar based on the fingerprints
    - Understand how the the similar ones can be used to aid the prediction


- Test more models
    - Current models have bad validation scores


In [1]:
import pickle

# Load the fingerprints dataset
with open('../data/mol_bits.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
print(type(data))
print(data)

---

In [2]:
from scipy.sparse import dok_matrix
import numpy as np

# Get sparse matrix size from the dataset
height = len(data.keys())
width = max( [max( list(data.values())[i] ) for i in range(height)] )

print(f"Height: {height}, Width: {width}")

# Lookup dictionaries for easier and fast access
mol_name_lookup = {mol_id: idx for idx, mol_id in enumerate(data.keys())}
mol_ids_lookup = {idx: mol_id for idx, mol_id in enumerate(data.keys())}

# Create the sparse matrix
mat = dok_matrix( (height,width), dtype=np.int32)

# Fill the matrix
for uniprot_idx, (_, struct_ids_list) in zip(range(height), data.items() ):
    for struct_id in struct_ids_list:

        # Indexs are starting from 1 in the dataset
        mat[uniprot_idx, struct_id-1 ] = 1

one_hot_arr = mat.toarray()

Height: 73865, Width: 2047


In [3]:
import matplotlib.pyplot as plt

# Code from TP05
def DrawSimPlot(B, R):
    S=np.arange(0,1.0,.01)
    v=(1/B)**(1/R)
    P=1-(1-S**R)**B
    plt.figure(figsize=(7,5))
    plt.plot(S,P)
    plt.axvline(x = v, linestyle ="--", color ='r')
    plt.title("Candidate pairs probability for B=%d and R=%d" % (B,R))
    plt.xlabel("Document Similarity")
    plt.ylabel("Probability of being a candidate pair")

    plt.grid()
    plt.show()

# Code from TP05
# THIS IS MODIFIED TO WORK WITH OUR DATA STRUCTURE
def MakeBucketsT(TDocs, perms, N,M, B, R, NB):
    Buckets={}
    all_docs=set(range(N))
    for b in range(B):
        SIGS=np.zeros((N, R), dtype="int32")           # initializes line sig
        for r in range(R):
            perm=perms[b*R+r]
            L=all_docs.copy()                         # gets all docs as a set
            i=0 
            while len(L)>0:
                elem=perm[i]                          # get new element  from permutation
                docs_found=TDocs[elem] & L            # get all the docs with a set bit on that elem that are still on the list
                if len(docs_found)>0:                 # if anything was found
                    SIGS[list(docs_found), r]=i       #   set the line sig to the current position from the perm
                    L=L-docs_found                    #   update the current list removing the found docs
                i+=1                                  # update the current position
                if i==M:                              #this is the case that the document is empty 
                    SIGS[list(L), r]=i                # Highly unlikely in a real data set  
                    L={}
                                                      # we have completed the signature for a given band, 
                                                      # now make the hashes for each document
        for d in range(N):
            bucket = hash(tuple(SIGS[d])) % NB
            Buckets.setdefault((b, bucket),set()).add(d)
    return Buckets

# Code from TP05
# THIS IS MODIFIED TO WORK WITH OUR DATA STRUCTURE
def LSHT(Data, B, R, N,M, NB=28934501 ):
    #transpose the data set
    
    DT=list(Data.values())

    DataT=[set(DT[i]) for i in range(M)]
    P=B*R
    np.random.seed(3)
    #print("Generating %d permutations for %6.3f similarity" %(P, (1/B)**(1/R)))
    perms=[np.random.permutation(M) for i in range(P)]
    buckets=MakeBucketsT(DataT, perms, N,M, B,R, NB)
    return buckets

def setify_similarity_results(band, current_dict, list_similar_names):

    for name in list_similar_names:
        for similar_name in list_similar_names:
            
            if similar_name != name:
                if name not in current_dict:
                    current_dict[name] = {band: set()} 
                
                if band not in current_dict[name]:
                    current_dict[name][band] = set()
                    
                current_dict[name][band].add(similar_name)

def JaccardSim(d1, d2):
    a =np.inner(d1,d2)
    bc=np.sum(d1+d2)-a
    return a/bc

In [17]:
s=0.7

for band_num in range(2000, 2001, 500):
    for row_num in range(2, 3):

        p = 1-(1-0.8**row_num)**band_num

        bucks = LSHT(data, band_num, row_num , M = height, N = width)

        # This dict will have a lot of redundancy, but it's easier to search for similar documents of a given document
        results_dict = {}

        for b, buck in bucks:
            if len(bucks[(b,buck)])>1:

                # get_names_from_ids = list(map(lambda x: protein_ids_lookup[x], bucks[(b,buck)]))

                # print("Band", b, "suggests these similar docs:", bucks[(b,buck)])

                setify_similarity_results(b, results_dict, bucks[(b,buck)])

        # ---

        # Check with Jaccard Sim
        # sim_above_threshold = []
        mol_similar = {}
        total_sim = 0
        num_pairs = 0

        for b, buck in bucks:

            if len(bucks[(b,buck)])>1:
                doc_ids=np.array(list(bucks[(b,buck)]))

                idx = np.stack(np.triu_indices(len(doc_ids), k=1), axis=-1)
                sim_pairs=doc_ids[idx]

                for mol1, mol2 in sim_pairs:
                    J=JaccardSim(one_hot_arr[mol1], one_hot_arr[mol2])


                    # print ("Jaccard Similarity between docs %d (%s) and %d (%s) is: %7.4f" %(d1, protein_ids_lookup[d1] , d2, protein_ids_lookup[d2] ,J), end="")
                    
                    if J > s: 
                        total_sim += J
                        num_pairs += 1

                        if mol1 not in mol_similar:
                            mol_similar[ mol_ids_lookup[mol1] ] = [ (mol_ids_lookup[mol2],J) ]
                        else:
                            mol_similar[ mol_ids_lookup[mol1] ].append( (mol_ids_lookup[mol2], J) )

                        # sim_above_threshold.append((d1, d2, J))

        # sim_above_threshold = sorted( set(sim_above_threshold), key=lambda x: x[2], reverse=True)

        # Sort within each dict entry
        for mol in mol_similar.keys():
            mol_similar[mol] = sorted(mol_similar[mol], key=lambda x: x[1], reverse=True) # tuple contains (mol, sim)


        # Print Jacc Sim above threshold
        print(f"Band: {band_num}, R: {row_num}, P {p} | Mean Jaccard Similarity: {total_sim/num_pairs} | Number of similar pairs: {num_pairs}")
        # for d1, d2, sim in sim_above_threshold:
        #     print(mol_ids_lookup[d1], mol_ids_lookup[d2], sim)
            # print(d1,d2, sim)

Band: 2000, R: 2, P 1.0 | Mean Jaccard Similarity: 0.7533402026986993 | Number of similar pairs: 3235


---

In [19]:
# Load the data from the csvs
import pandas as pd
import numpy as np
import random

random.seed(42)

df_train_original = pd.read_csv("../data/activity_train.csv", header=None)
df_test = pd.read_csv("../data/activity_test_blanked.csv", header=None)


num_lines_train = len(df_train_original)

# Merge (Why? In order to not have overlapping mappings in the future)
df_merged = pd.concat([df_train_original, df_test], axis=0, ignore_index=True)


In [20]:
# Pre-Processing
from sklearn.preprocessing import LabelEncoder

# Get the input arrays from the df
X_full_labels_x1 = df_merged.iloc[:,0].to_numpy()
X_full_labels_x2 = df_merged.iloc[:,1].to_numpy()

# Get unique values for x1
label_enc_x1 = LabelEncoder()
label_enc_x1.fit(X_full_labels_x1)
X_full_transformed_x1 = label_enc_x1.transform(X_full_labels_x1)

# Get unique values for x2
label_enc_x2 = LabelEncoder()
label_enc_x2.fit(X_full_labels_x2)
X_full_transformed_x2 = label_enc_x2.transform(X_full_labels_x2)

# Revert back into train and test
X_train_transformed_x1 = X_full_transformed_x1[:num_lines_train]
X_train_transformed_x2 = X_full_transformed_x2[:num_lines_train]

X_test_transformed_x1 = X_full_transformed_x1[num_lines_train:]
X_test_transformed_x2 = X_full_transformed_x2[num_lines_train:]


# Convert the arrays to dataframe again
df_train_transformed = pd.DataFrame(data = np.column_stack((X_train_transformed_x1, X_train_transformed_x2, df_train_original.iloc[:,2].to_numpy())), columns = [0,1,2])

# Split train into train and validation
train_idxs = random.sample(range(0, num_lines_train), int(num_lines_train * 0.9))

df_train = df_train_transformed.iloc[train_idxs]
df_val = df_train_transformed.drop(train_idxs)


X_train_transformed_x1 = df_train.iloc[:,0].to_numpy()
X_train_transformed_x2 = df_train.iloc[:,1].to_numpy()

X_val_transformed_x1 = df_val.iloc[:,0].to_numpy()
X_val_transformed_x2 = df_val.iloc[:,1].to_numpy()



In [21]:
# Get final inputs for the model
X_train = np.column_stack((X_train_transformed_x1, X_train_transformed_x2))
Y_train = df_train.iloc[:,2].to_numpy()

X_val = np.column_stack((X_val_transformed_x1, X_val_transformed_x2))
Y_val = df_val.iloc[:,2].to_numpy()

X_test = np.column_stack((X_test_transformed_x1, X_test_transformed_x2))

In [51]:
# THIS CODE IS ONLY IF WE WANT TO USE THE MOLECULES STRUCTURES TO TRY TO IMPROVE THE MODEL
# Use the molecules structures to see if it helps the model perform better
MAX_SIMILAR_MOLS = 3

# First, add padding to the training matrix to handle more inputs (-1 because it does not correspond to any molecule)
X_train = np.pad( X_train, ((0, 0), (0, MAX_SIMILAR_MOLS)), mode='constant', constant_values=-1)

X_train_mol_names = label_enc_x2.inverse_transform(X_train[:,1])

# Iterate over the input molecules and check if we have found similar molecules in the LSH
# If we have, we will add the similarity to the input data
mols = list(mol_similar.keys())

# TODO: fit the molecule names to the label encoder, some of them may not be present

for idx_train, mol in enumerate(X_train_mol_names):

    # print("Processing ", mol, " - ", idx_train)
    # Check if we have found similar molecules
    mol = mol.strip(" ")

    if mol in mols:
        print("Found similar molecules for ", mol)

        # Add the similar molecules to the input data
        for i, (mol2, sim) in enumerate(mol_similar[mol]):

            print("Similar molecule: ", mol2, " with similarity: ", sim)
            mol2 = mol2.strip(" ")

            if i < MAX_SIMILAR_MOLS:
                # Get the index (mapping) of the molecule
                mol2_label = label_enc_x2.transform([mol2])[0]

                # Put in the input matrix
                X_train[idx_train][2+i] = mol2_label
            else:
                break


Found similar molecules for  CHEMBL3649211
Similar molecule:  CHEMBL3649232  with similarity:  0.7073170731707317


ValueError: y contains previously unseen labels: 'CHEMBL3649232'

In [46]:
print(mols)

['CHEMBL3663552', 'CHEMBL3649180', 'CHEMBL3652441', 'CHEMBL3704949', 'CHEMBL3670539', 'CHEMBL3672938', 'CHEMBL3586419', 'CHEMBL3649136', 'CHEMBL182473', 'CHEMBL3670631', 'CHEMBL44542', 'CHEMBL4084072', 'CHEMBL3426137', 'CHEMBL3663555', 'CHEMBL1830958', 'CHEMBL4798993', 'CHEMBL4089496', 'CHEMBL3672936', 'CHEMBL33827', 'CHEMBL4086564', 'CHEMBL2385133', 'CHEMBL3665636', 'CHEMBL3649169', 'CHEMBL2047164', 'CHEMBL4099781', 'CHEMBL4080914', 'CHEMBL4083587', 'CHEMBL3586414', 'CHEMBL3646170', 'CHEMBL233560', 'CHEMBL3646197', 'CHEMBL3649143', 'CHEMBL4084277', 'CHEMBL3649217', 'CHEMBL3649153', 'CHEMBL3585948', 'CHEMBL3652530', 'CHEMBL3649129', 'CHEMBL3639624', 'CHEMBL4786299', 'CHEMBL387446', 'CHEMBL3669457', 'CHEMBL3663488', 'CHEMBL3585951', 'CHEMBL3663551', 'CHEMBL3586425', 'CHEMBL3663575', 'CHEMBL3649140', 'CHEMBL4105072', 'CHEMBL3426144', 'CHEMBL3659200', 'CHEMBL3585949', 'CHEMBL1271490', 'CHEMBL214935', 'CHEMBL4079180', 'CHEMBL3669473', 'CHEMBL3691810', 'CHEMBL3426143', 'CHEMBL4077366', 'CHE

In [36]:
print(np.where(X_train[:,2:] != -1))

(array([], dtype=int64), array([], dtype=int64))


In [24]:
print(X_train_mol_names.shape)
print(X_train.shape)
print(X_train[:20])

(122139,)
(122139, 5)
[[   19 17121    -1    -1    -1]
 [    8 72350    -1    -1    -1]
 [   63 49119    -1    -1    -1]
 [   51 27651    -1    -1    -1]
 [   46 60276    -1    -1    -1]
 [   25 20478    -1    -1    -1]
 [   16 60695    -1    -1    -1]
 [   16 25435    -1    -1    -1]
 [   88 13749    -1    -1    -1]
 [   10 11389    -1    -1    -1]
 [   10 35903    -1    -1    -1]
 [   16 25710    -1    -1    -1]
 [   46 34423    -1    -1    -1]
 [   47 56089    -1    -1    -1]
 [  141  2049    -1    -1    -1]
 [    9 54529    -1    -1    -1]
 [   42 49010    -1    -1    -1]
 [   86 59770    -1    -1    -1]
 [   46 49983    -1    -1    -1]
 [  108 53299    -1    -1    -1]]


In [6]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_classifier_model(model, X, Y):
    Y_pred = model.predict(X)
    acc = accuracy_score(Y, Y_pred)
    f1 = f1_score(Y, Y_pred, average="macro")
    precision = precision_score(Y, Y_pred, average="macro", zero_division=0)
    recall = recall_score(Y, Y_pred, average="macro")
    matt = matthews_corrcoef(Y, Y_pred)

    return acc, f1, precision, recall, matt

def print_evaluation_classifier_results(acc, f1, precision, recall, matt, model_name = None):
    print("--------------------")
    if model_name is not None:
        print(f"Results for model: {model_name}")
    print(f"Accuracy: {acc}")
    print(f"F1: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Matthews Correlation Coefficient: {matt}")



def evaluate_regression_model(model, X, Y):
    Y_pred = model.predict(X)
    mse = mean_squared_error(Y, Y_pred)
    mae = mean_absolute_error(Y, Y_pred)
    r2 = r2_score(Y, Y_pred)

    return mse, mae, r2

def print_evaluation_regression_results(mse, mae, r2, model_name = None):
    print("--------------------")
    if model_name is not None:
        print(f"Results for model: {model_name}")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R2: {r2}")

In [31]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVC
import copy


import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

# --- SVM ---
# model = SVC(random_state=42)
# model.fit(X_train, Y_train)

# metrics = evaluate_classifier_model(model, X_val, Y_val)
# print_evaluation_classifier_results(*metrics, model_name = "SVM")


# --- Random Forest Regressor ---
# num_estimators = range(10, 110, 10)
# val_scores = []
# best_model_score = np.inf
# best_model = None

# # Grid search for the best number of estimators
# for i in num_estimators:

#     # Train the model
#     model = RandomForestClassifier(n_estimators=i, random_state=42)
#     model.fit(X_train, Y_train)

#     # Calculate model error on validation set
#     metrics = evaluate_classifier_model(model, X_val, Y_val)
#     print_evaluation_classifier_results(*metrics, model_name = f"RFR with {i} trees")

#     val_scores.append(metrics[0])
#     # val_scores.append(model.score(X_val, Y_val))
#     # print(f"MLP with {layers} layers has a validation score of {val_scores[-1]}")

#     # Save the best model
#     if val_scores[-1] < best_model_score:
#         best_model_score = val_scores[-1]
#         best_model = model

# # Plot the validation scores for each number of estimators
# plt.plot(num_estimators, val_scores)
# plt.xlabel("Number of estimators")
# plt.ylabel("Validation score (MSE)")
# plt.show()

# Get the predictions
# rfr_predictions = model_rfr.predict(X_test)


# --- Linear Regression ---
# lr = LinearRegression(n_jobs=-1)
# lr.fit(X_train, Y_train)

# Calculate model error on validation set
# metrics = evaluate_regression_model(lr, X_val, Y_val)
# print_evaluation_regression_results(*metrics, model_name = "Linear Regression")

# # # Get the predictions
# # lr_predictions = lr.predict(X_test)


# # --- MLP Regression ---

# hidden_layer_shapes = [(100), (150), (200, 100), (200, 200), (200, 200, 200), (200, 200, 300), (200, 250, 300, 150)]

# val_scores = []
# best_model_score = np.inf
# lr = 1e-3

# # Grid search for the best number of hidden layers
# for layers in hidden_layer_shapes:
    
#         # Train the model
#         model = MLPRegressor(hidden_layer_sizes=layers, learning_rate_init=lr, activation="relu", batch_size = 64 , random_state=42)
#         model.fit(X_train, Y_train)
    
#         # Calculate model error on validation set
#         metrics = evaluate_regression_model(model, X_val, Y_val)
#         print_evaluation_regression_results(*metrics, model_name = f"MLP Regressor with {layers} hidden layers")

#         val_scores.append(metrics[0])
#         # val_scores.append(model.score(X_val, Y_val))
#         # print(f"MLP with {layers} layers has a validation score of {val_scores[-1]}")
    
#         # Save the best model
#         if val_scores[-1] < best_model_score:
#             best_model_score = val_scores[-1]
#             best_model = model

# --- MLP Classifier ---
hidden_layer_shapes = [(100)]#, (150), (200, 100), (200, 200), (200, 200, 200), (200, 200, 300), (200, 250, 300, 150)]

val_scores = []
best_model_score = 0
lr = 1e-3

TEST_NUM = 1000

# Grid search for the best number of hidden layers
# for layers in hidden_layer_shapes:

#         # Train the model
#         model = MLPClassifier(hidden_layer_sizes=layers, learning_rate_init=lr, activation="relu", batch_size = 64 , random_state=42)
#         model.fit(X_train[:TEST_NUM], Y_train[:TEST_NUM])
    
#         # Calculate model error on validation set
#         metrics = evaluate_classifier_model(model, X_val, Y_val)
#         print_evaluation_classifier_results(*metrics, model_name = f"MLP Classifier with {layers} hidden layers")

#         val_scores.append(metrics[0])
#         # val_scores.append(model.score(X_val, Y_val))
#         # print(f"MLP with {layers} layers has a validation score of {val_scores[-1]}")
    
#         # Save the best model
#         if val_scores[-1] > best_model_score:
#             best_model_score = val_scores[-1]
#             best_model = model


(10, 2)
(10, 5)
[[   19 17121    -1    -1    -1]
 [    8 72350    -1    -1    -1]
 [   63 49119    -1    -1    -1]
 [   51 27651    -1    -1    -1]
 [   46 60276    -1    -1    -1]
 [   25 20478    -1    -1    -1]
 [   16 60695    -1    -1    -1]
 [   16 25435    -1    -1    -1]
 [   88 13749    -1    -1    -1]
 [   10 11389    -1    -1    -1]]
